# Tracking Transformers in [Mojito](https://www.youtube.com/watch?v=-biOGdYiF-I) <a href="https://colab.research.google.com/github/eto-ai/rikai/blob/main/notebooks/MojitoVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 麻烦给我的爱人来一杯 Mojito 🎵🎵🎵

This notebook demonstrates how to track transformers. [spark-video](https://github.com/eto-ai/spark-video) is required.

![](https://canvas-bridge02.tubitv.com/exp01/Q7N97TP4AKS3COmXOGU4Qo4OVwY=/400x574/smart/img.adrise.tv/ec2e6d2f-538b-4b74-997a-4e56c3783676.png)

In [ ]:
# This paragraph is for Google Colab
# Please skip it if you have installed all these python packages and jars locally

# python packages
!pip install ipython==7.31.1
!pip install rikai
!pip install rikai-torchhub
!pip install rikai-yolov5

# jars
!wget -O /usr/local/lib/python3.7/dist-packages/pyspark/jars/ffmpeg-4.4-1.5.6-linux-x86_64.jar https://repo1.maven.org/maven2/org/bytedeco/ffmpeg/4.4-1.5.6/ffmpeg-4.4-1.5.6-linux-x86_64.jar
!wget -O /usr/local/lib/python3.7/dist-packages/pyspark/jars/javacpp-1.5.6-linux-x86_64.jar https://repo1.maven.org/maven2/org/bytedeco/javacpp/1.5.6/javacpp-1.5.6-linux-x86_64.jar
!wget -O /usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-video-assembly-0.0.4.jar https://github.com/eto-ai/spark-video/releases/download/v0.0.4/spark-video-assembly_2.12-0.0.4.jar

In [ ]:
!wget https://github.com/da-tubi/rikai-example/raw/main/elephants_dream.mp4

## Video Visualization

In [5]:
from rikai.types.video import VideoStream
uri = 'https://user-images.githubusercontent.com/1267865/154184628-d635c62e-d441-4853-8a67-65a0e838157d.mp4'
video = VideoStream(uri)
video

VideoStream(uri=https://user-images.githubusercontent.com/1267865/154184628-d635c62e-d441-4853-8a67-65a0e838157d.mp4)

## Initialization
+ Initialize the Spark Session with Rikai support
+ Register all built-in UDFs
+ Create the yolov5m model

In [1]:
from rikai.spark.utils import init_spark_session
from rikai.spark.functions import init

spark = init_spark_session(
    dict(
        [
            (
                "spark.rikai.sql.ml.registry.torchhub.impl",
                "ai.eto.rikai.sql.model.torchhub.TorchHubRegistry",
            )
        ]
    )
)

init(spark)

In [2]:
spark.sql("""
CREATE MODEL yolov5m
OPTIONS (device="cpu", batch_size=32)
USING "torchhub:///ultralytics/yolov5:v6.0/yolov5m";
""")

DataFrame[]

In [3]:
spark.sql(f"""
select count(*)
from video.`/content/elephants_dream.mp4`
""").toPandas()

,count(1)
0,15691


## How many objects in this video

In [7]:
spark.sql(f"""
create or replace temporary view elephants_dream
using video
options (
  path '/content/elephants_dream.mp4',
  frameStepSize 100
)
""")

DataFrame[]

In [8]:
df = spark.sql(f"""
select pred.label, count(*)
from (
  select *, explode(ML_PREDICT(yolov5m, image)) as pred
  from (
    select to_image(image_data) as image
    from elephants_dream
  )
)
group by pred.label
""")
df.toPandas()

,label,count(1)
0,motorcycle,1
1,handbag,1
2,person,123
3,skateboard,5
4,cell phone,1
5,train,1
6,vase,1
7,surfboard,1
8,bird,1
9,wine glass,2


## Hey, let me see if the motocycles and trains are transformers?

In [15]:
cases = spark.sql(f"""
select frame_id, pred.label, pred.box, image
from (
  select *, explode(ML_PREDICT(yolov5m, image)) as pred
  from (
    select frame_id, to_image(image_data) as image
    from elephants_dream
  )
)
where
  pred.label in ('train', 'motorcycle')
""").toPandas()

cases

,frame_id,label,box,image
0,10100,motorcycle,"(297.9526062011719, 57.09489059448242, 424.968...",Image(<embedded>)
1,10700,train,"(5.7795090675354, 24.075878143310547, 424.6746...",Image(<embedded>)


## Ooops, not a motorcycle?! Is it a transformer?

In [17]:
cases.image[0] | cases.box[0]

Draw(Box2d(xmin=297.9526062011719, ymin=57.09489059448242, xmax=424.96820068359375, ymax=236.07225036621094))

## Oh no, we are eaten by transformers!

In [18]:
cases.image[1] | cases.box[1]

Draw(Box2d(xmin=5.7795090675354, ymin=24.075878143310547, xmax=424.6746520996094, ymax=234.36643981933594))